In [1]:
import json
import os
import random
import shap
import torch
import math
import torch.nn as nn 
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from torch import optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
import time
import warnings
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
warnings.filterwarnings("ignore")
import re



from transformers import BertTokenizer, BertModel, XLMTokenizer, XLMConfig, XLMModel, \
    AlbertTokenizer, AlbertModel, RobertaTokenizer, RobertaConfig, RobertaModel


Model

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


Choose LLM

In [3]:
def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [atoi(c) for c in re.split('(\d+)',text) ]

In [4]:
LLM = "bert-base-uncased"
# LLM = "bert-large-uncased"
# LLM = "albert"
# LLM = "roberta"
# LLM = "xlm"

def change_llm(LLM):

    if "bert-" in LLM:
        tokenizer = BertTokenizer.from_pretrained(LLM)
        llm = BertModel.from_pretrained(LLM)


    if LLM == "albert":
        tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
        llm = AlbertModel.from_pretrained('albert-base-v2')

    if LLM == "roberta":
        tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
        configuration = RobertaConfig()
        llm = RobertaModel(configuration)

    if LLM == "xlm":
        tokenizer = XLMTokenizer.from_pretrained("xlm-mlm-en-2048")
        configuration = XLMConfig()
        llm = XLMModel(configuration)


    llm.eval()
    sentence = "Paris is a beautiful city" 
    tok = tokenizer(sentence)
    encoded_layers = llm(torch.Tensor([tok.input_ids]).to(torch.int), attention_mask=torch.Tensor([tok.attention_mask]))
    llm_size = (encoded_layers.last_hidden_state)[:, 0, :].size()[1]

    return tokenizer, llm, llm_size
tokenizer, llm, llm_size = change_llm(LLM)

In [8]:
lv1_lv3 = dict({0:0, 1:0, 2:0, 3:0, 4:0, 5:1, 6:1, 7:1, 8:1, 9:1, 10:1, 11:1, 12:1, 13:2, 14:2, 15:2, 16:2, 17:2, 18:2})

In [9]:
class DATA():

    def __init__(self):
        self.dataset_llm = []
        self.dataset_opensmile = []
        self.dataset_cps_f = []
        self.dataset_cps_i = []
        self.dataset_action = []
        self.dataset_gamr = []

        self.targets = []

    def openLLM(self, filename):
        data = pd.read_csv(filename)
        for i in range(len(data)):
            self.dataset_llm.append(np.asarray(data.iloc[i]).tolist())
    
    
    def openSmile(self,filename):
        data = pd.read_csv(filename)
        for i in range (data.shape[0]):
            row=data[data['file']==f'C:\\Users\\bradf\\OneDrive - Colostate\\Research\\Initial Observations for Fib Weights\\Data\\Segment Analysis\\{filename[filename.index("Group_"):filename.index("Group_")+8]}\\segments_oracle\\{filename[filename.index("Group_"):filename.index("Group_")+8]}_'+str(i)+'.wav']
            tensor=np.asarray(row.values[0][3:91],dtype=np.float32).tolist()
            self.dataset_opensmile.append(tensor)


    def openCPS(self, filename):
        data = pd.read_csv(filename).fillna(0)
        for row in range(data.shape[0]):
            cps = data.iloc[row, 8:].fillna(0)
            self.dataset_cps_i.append(list(cps.values))
            cps_f = [0, 0, 0]
            for i, c in enumerate(list(cps.values)):
                if c == 1:
                    cps_f[lv1_lv3[i]] = 1
            self.dataset_cps_f.append(cps_f)

    def openAction(self, filename):
        data = pd.read_csv(filename, header=None).fillna(0)
        for row in range(data.shape[0]):
            self.dataset_action.append(data.iloc[row].to_list())



    def openGAMR(self, filename):
        data = pd.read_csv(filename, header=None).fillna(0)
        for row in range(data.shape[0]):
            self.dataset_gamr.append(data.iloc[row].to_list())
        

    def openTarget(self,filename):
        data = pd.read_csv(filename)
        for row in range(data.shape[0]):
            target = data.iloc[row, 4:].values.astype(int)
            self.targets.append(target.tolist())


    def get_datasets(self, rand=False):
        final_dataset=[]        
        for llm, opensmile, cps_f, cps_i, action, gamr, label in zip(self.dataset_llm, self.dataset_opensmile,self.dataset_cps_f, self.dataset_cps_i, self.dataset_action, self.dataset_gamr, self.targets):
            final_dataset.append([llm, opensmile, cps_f, cps_i, action, gamr, label])
            
        if rand:
            random.shuffle(final_dataset)
        return final_dataset

In [10]:
def read_data(dataset, root, file, llm="bert-base-uncased"):

    if LLM in file:
        dataset.openLLM(root+"/"+file)
    if 'oracle_opensmile' in file:
        dataset.openSmile(root+"/"+file)
    elif 'CPS_Oracle' in file:
        dataset.openCPS(root+"/"+file)
    elif "Actions_Oracle_vectors" in file:
        dataset.openAction(root+"/"+file)
    elif "GAMR_Oracle_vectors" in file:
        dataset.openGAMR(root+"/"+file)
    elif 'CG.csv' in file:
        dataset.openTarget(root+"/"+file)
        

In [11]:
train_datasets = DATA()

In [ ]:
for root, dirs,files in (os.walk(os. getcwd())):
    if "Group_" in root:
        if "asr" not in root:
            print(root)
            for file in files:
                read_data(train_datasets, root, file)

In [13]:
len(train_datasets.dataset_llm) == len(train_datasets.dataset_opensmile) == len(train_datasets.dataset_cps_f) == len(train_datasets.dataset_cps_i) == len(train_datasets.dataset_action) == len(train_datasets.dataset_gamr) == len(train_datasets.targets)

True

In [15]:
# scaler = MinMaxScaler()
# train_datasets.dataset_llm = scaler.fit_transform(train_datasets.dataset_llm).tolist()
# scaler = MinMaxScaler()
# train_datasets.dataset_opensmile = scaler.fit_transform(train_datasets.dataset_opensmile).tolist()

In [19]:
l = train_datasets.get_datasets()
l = [a+b+c+d+e+f for a,b,c,d,e,f,g in l]

In [20]:
n_folds = 3
fold_len = int(np.ceil(len(train_datasets.get_datasets())/n_folds))
fold_len

958

In [21]:
folds = [train_datasets.get_datasets()[x:x+fold_len] for x in range(0, len(train_datasets.get_datasets()), fold_len)] 

Recurrent model

In [22]:
train_list = train_datasets.get_datasets()

In [23]:
train_list = [[a+b+c+d+e+f, g] for a,b,c,d,e,f,g in train_list]

variable window size

In [92]:
window_size = 7

In [105]:
def change_window_size(window_size):
    rec_train_list = []
    pad = [[0]*1199, [0]*7]

    for utt_id in range(len(train_list)):
        aux = []
        for i in range(window_size):
            if utt_id == i:
                for _ in range(window_size-utt_id):
                    aux.append(pad)
        for i in range(window_size):
            if len(aux) == i:
                aux.append(train_list[utt_id - window_size + i])
        aux.append(train_list[utt_id])
        rec_train_list.append(aux)
    
    return rec_train_list

In [107]:
rec_train_list = change_window_size(window_size)

In [26]:
llm_size+88+19+3+78+243

1199

In [94]:
class rec_dataset(Dataset):
    def __init__(self,xy=None):
        self.utt_bert_l = []
        self.utt_open_l = []
        self.utt_cps_f_l = []
        self.utt_cps_i_l = []
        self.utt_action_l = []
        self.utt_gamr_l = []
        self.utt_y_l = []

        for utt in xy:
            bert_tmp = []
            open_tmp = []
            cps_i_tmp = []
            cps_f_tmp = []
            action_tmp = []
            gamr_tmp = []
            y_tmp = []
            for utt_id in range(window_size+1):
                bert_tmp.append(utt[utt_id][0][:llm_size])
                open_tmp.append(utt[utt_id][0][llm_size:llm_size+89])
                cps_i_tmp.append(utt[utt_id][0][llm_size+89:llm_size+89+19])
                cps_f_tmp.append(utt[utt_id][0][llm_size+89+19:llm_size+89+20+3])
                action_tmp.append(utt[utt_id][0][llm_size+89+20+3:llm_size+89+20+3+78])
                gamr_tmp.append(utt[utt_id][0][llm_size+89+20+3+78:])
                y_tmp.append(utt[utt_id][1])
            self.utt_bert_l.append(bert_tmp)
            self.utt_open_l.append(open_tmp)
            self.utt_cps_f_l.append(cps_i_tmp)
            self.utt_cps_i_l.append(cps_f_tmp)
            self.utt_action_l.append(gamr_tmp)
            self.utt_gamr_l.append(action_tmp)
            self.utt_y_l.append(y_tmp)
        self.utt_bert_l = torch.from_numpy(np.asarray(self.utt_bert_l))
        self.utt_open_l = torch.from_numpy(np.asarray(self.utt_open_l))
        self.utt_cps_f_l = torch.from_numpy(np.asarray(self.utt_cps_f_l))
        self.utt_cps_i_l = torch.from_numpy(np.asarray(self.utt_cps_i_l))
        self.utt_action_l = torch.from_numpy(np.asarray(self.utt_action_l))
        self.utt_gamr_l = torch.from_numpy(np.asarray(self.utt_gamr_l))
        self.utt_y_l = torch.from_numpy(np.asarray(self.utt_y_l))



        self.len=len(self.utt_bert_l)
        

    def __getitem__(self, index):
        return self.utt_bert_l[index], self.utt_open_l[index], self.utt_cps_f_l[index], self.utt_cps_i_l[index], self.utt_action_l[index], self.utt_gamr_l[index], self.utt_y_l[index]
    
    def __len__(self):
        return self.len

In [33]:
rec_train_loader = DataLoader(dataset=rec_dataset(rec_train_list),batch_size=16,shuffle=False)

In [57]:
class rec_common_ground(nn.Module):
    def __init__(self, lin_layers=True, bert_b=True, opensmile_b=True, cps_b=True, cps_f_b=True, action_b=True, gamr_b=True, output_size=1):
        super(rec_common_ground, self).__init__()
        self.lin_bert1 = nn.Linear(llm_size, 256)
        self.lin_bert2 = nn.Linear(256, 256)
        self.lstm_bert = nn.LSTM(input_size=256, batch_first=True, hidden_size=256)
        if not lin_layers:
            self.lstm_bert = nn.LSTM(input_size=llm_size, batch_first=True, hidden_size=256)


        self.lin_open1 = nn.Linear(88, 256)
        self.lin_open2 = nn.Linear(256, 256)
        self.lstm_opensmile = nn.LSTM(input_size=256, batch_first=True, hidden_size=256)
        if not lin_layers:
            self.lstm_opensmile = nn.LSTM(input_size=88, batch_first=True, hidden_size=256)
        self.relu = nn.ReLU()

        cps_size = 19
        if cps_f_b:
            cps_size = 3
        self.lin_cps1 = nn.Linear(cps_size, 256)
        self.lin_cps2 = nn.Linear(256, 256)
        self.lstm_cps = nn.LSTM(input_size=256, batch_first=True, hidden_size=256)
        if not lin_layers:
            self.lstm_cps = nn.LSTM(input_size=cps_size, batch_first=True, hidden_size=256)


        self.lin_action1 = nn.Linear(78, 256)
        self.lin_action2 = nn.Linear(256, 256)
        self.lstm_action = nn.LSTM(input_size=256, batch_first=True, hidden_size=256)
        if not lin_layers:
            self.lstm_action = nn.LSTM(input_size=78, batch_first=True, hidden_size=256)

        self.lin_gamr1 = nn.Linear(243, 256)
        self.lin_gamr2 = nn.Linear(256, 256)
        self.lstm_gamr = nn.LSTM(input_size=256, batch_first=True, hidden_size=256)
        if not lin_layers:
            self.lstm_gamr = nn.LSTM(input_size=243, batch_first=True, hidden_size=256)



        n_modals = int(bert_b) + int(opensmile_b) + int(cps_b) + int(action_b) + int(gamr_b)
        self.ff1 = nn.Linear(256*n_modals, 512)
        self.ff2 = nn.Linear(512, 512)
        self.act1 = nn.Tanh()
        self.act2 = nn.SiLU()
        self.classifier = nn.Linear(512, output_size)


    def forward(self, utt_bert_l, utt_open_l, utt_cps_l, utt_action_l, utt_gamr_l, lin_layers=True, bert_b=True, opensmile_b=True, cps_b=True, cps_f_b=True, action_b=True, gamr_b=True):

        if bert_b:
            if lin_layers:
                utt_bert_l = [self.lin_bert1(utt_x_bert) for utt_x_bert in utt_bert_l]
                utt_bert_l = [self.lin_bert2(utt_x_bert) for utt_x_bert in utt_bert_l]
                utt_bert_l = [self.relu(utt_x_bert) for utt_x_bert in utt_bert_l]
            bert = torch.stack(tuple(utt_bert_l), dim=1)
            bert = self.lstm_bert(bert)[1][0][0]

        if opensmile_b:
            if lin_layers:
                utt_open_l = [self.lin_open1(utt_x_open) for utt_x_open in utt_open_l]
                utt_open_l = [self.lin_open2(utt_x_open) for utt_x_open in utt_open_l]
                utt_open_l = [self.relu(utt_x_open) for utt_x_open in utt_open_l]
            opensmile = torch.stack(tuple(utt_open_l), dim=1)
            opensmile = self.lstm_opensmile(opensmile)[1][0][0]

        if cps_b:
            if lin_layers:
                utt_cps_l = [self.lin_cps1(utt_x_cps) for utt_x_cps in utt_cps_l]
                utt_cps_l = [self.lin_cps2(utt_x_cps) for utt_x_cps in utt_cps_l]
                utt_cps_l = [self.relu(utt_x_cps) for utt_x_cps in utt_cps_l]
            cps = torch.stack(tuple(utt_cps_l), dim=1)
            cps = self.lstm_cps(cps)[1][0][0]

        if action_b:
            if lin_layers:
                utt_action_l = [self.lin_action1(utt_x_action) for utt_x_action in utt_action_l]
                utt_action_l = [self.lin_action2(utt_x_action) for utt_x_action in utt_action_l]
                utt_action_l = [self.relu(utt_x_action) for utt_x_action in utt_cps_l]
            action = torch.stack(tuple(utt_action_l), dim=1)
            action = self.lstm_action(action)[1][0][0]

        if gamr_b:
            if lin_layers:
                utt_gamr_l = [self.lin_gamr1(utt_x_gamr) for utt_x_gamr in utt_gamr_l]
                utt_gamr_l = [self.lin_gamr2(utt_x_gamr) for utt_x_gamr in utt_gamr_l]
                utt_gamr_l = [self.relu(utt_x_gamr) for utt_x_gamr in utt_gamr_l]
            gamr = torch.stack(tuple(utt_gamr_l), dim=1)
            gamr = self.lstm_gamr(gamr)[1][0][0]

        modals = []
        if bert_b: modals.append(bert)
        if opensmile_b: modals.append(opensmile)
        if cps_b: modals.append(cps)
        if action_b: modals.append(action)
        if gamr_b: modals.append(gamr)


        x = torch.hstack(tuple(modals))
        x = self.ff1(x)
        x = self.act1(x)
        x = self.ff2(x)
        x = self.act2(x)
        predict = self.classifier(x)

        return predict


In [171]:
def rec_train(model, total_epochs, lr, train_iterator, class_to_eval, lin_layers, bert_b, opensmile_b, cps_b, cps_f_b, action_b, gamr_b, output_size):
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    epoch_loss = []
    nepochs = 0
    model = model.to(device)
    model.train()
    criterion = nn.BCELoss(reduction='mean').to(device)
    if output_size != 1:
        criterion = nn.CrossEntropyLoss(reduction='mean').to(device)
    while nepochs < total_epochs :
        optimizer.zero_grad()
        batch_loss = []
        for batch_idx, (utt_bert_l, utt_open_l, utt_cps_f_l, utt_cps_i_l, utt_action_l, utt_gamr_l, utt_y_l) in enumerate(train_iterator):
            utt_cps_l = utt_cps_f_l
            if not cps_f_b:
                utt_cps_l = utt_cps_i_l
            output = model(utt_bert_l, utt_open_l, utt_cps_l, utt_action_l, utt_gamr_l, lin_layers, bert_b, opensmile_b, cps_b, cps_f_b, action_b, gamr_b)
            target = utt_y_l[-1]
            if output_size == 1:
                target_binary = torch.zeros(target.size()[0], 1).to(device)
                for i,t in enumerate(target):
                    target_binary[i] = torch.Tensor([t[class_to_eval]])
                loss = criterion(torch.sigmoid(output).to(device), target_binary)
            else:
                loss = criterion(output, target)
            batch_loss.append(loss)
            loss.backward()
            optimizer.step()
        nepochs += 1
        epoch_loss.append(sum(batch_loss)/len(batch_loss))
    return model, epoch_loss

In [172]:
def rec_test(model, test_iterator, class_to_eval, lin_layers, bert_b, opensmile_b, cps_b, cps_f_b, action_b, gamr_b, output_size):
    with torch.no_grad():
        model.eval()
        true, pred = None, None
        for batch_idx, (utt_bert_l, utt_open_l, utt_cps_f_l, utt_cps_i_l, utt_action_l, utt_gamr_l, utt_y_l) in enumerate(test_iterator):
            utt_cps_l = utt_cps_f_l
            if not cps_f_b:
                utt_cps_l = utt_cps_i_l
            output = model(utt_bert_l, utt_open_l, utt_cps_l, utt_action_l, utt_gamr_l, lin_layers, bert_b, opensmile_b, cps_b, cps_f_b, action_b, gamr_b)
            target = utt_y_l[-1]
            if output_size == 1:
                pred_tmp = torch.sigmoid(output)
                true_binary = torch.zeros(target.size()[0], 1)
                for i, t in enumerate(target):
                    true_binary[i] = torch.Tensor([t[class_to_eval]])
                if true == None:
                    true = true_binary
                    pred = pred_tmp > 0.5
                    pred_probs = pred_tmp

                else :
                    true = torch.cat((true, true_binary))
                    pred = torch.cat((pred, pred_tmp > 0.5))
                    pred_probs = torch.cat((pred_probs, pred_tmp))
                    
            
            else:
                softmax = torch.nn.Softmax(dim=1)
                target_ = torch.clone(target)
                preds_tmp = softmax(output).cpu().detach().numpy()
                true_tmp = target_.cpu().numpy()

                if true == None:
                    true = torch.argmax(torch.from_numpy(true_tmp), dim=1)
                    pred = torch.argmax(torch.from_numpy(preds_tmp), dim=1)
                    preds_probs = torch.tensor(preds_tmp)
                else:
                    true = torch.cat((true, torch.argmax(torch.from_numpy(true_tmp), dim=1)))
                    pred = torch.cat((pred, torch.argmax(torch.from_numpy(preds_tmp), dim=1)))
                    preds_probs = torch.cat((preds_probs, torch.tensor(preds_tmp)))
        
    return true, pred, pred_probs


In [ ]:
LLMs = ["bert-base-uncased", "bert-large-uncased", "albert", "roberta", "xlm"]
n_folds = [5, 6, 7] # train test valid
window_sizes = [1, 3, 5, 7, 9]
lrs = [0.01, 0.001, 0.0001]
output_sizes = [1, 8, 9] # [binary, multinomial without negatives, multinomial with all utterances]
bert_b = [True, False]
opensmile_b = [True, False]
action_b = [True, False]
gamr_b = [True, False]
cps_b = [True, False]
cps_f_b = [True, False]
lin_layers = [True, False]


In [ ]:
tokenizer, llm, llm_size = change_llm(LLM)
fold_len = int(np.ceil(len(train_datasets.get_datasets())/n_folds))
folds = [train_datasets.get_datasets()[x:x+fold_len] for x in range(0, len(train_datasets.get_datasets()), fold_len)]

for class_to_eval in range(6):
    print(f"Results for class {class_to_eval}")
    for k in range(len(folds)):
        train_l = []
        for i in range(len(folds)):
            if i != k:
                train_l += folds[i]
        test_l = folds[k]
        train_loader = DataLoader(dataset=rec_dataset(train_l),batch_size=16,shuffle=False)
        test_loader = DataLoader(dataset=rec_dataset(test_l),batch_size=16,shuffle=False)
        
        
        
        model = rec_common_ground(lin_layers, bert_b, opensmile_b, output_size).to(device)
        rec_train(model, 60, train_loader, class_to_eval)
        true, pred, pred_probs = rec_test(model, test_loader, class_to_eval)





        # try:
        #     auroc = roc_auc_score(true, torch.nan_to_num(pred_probs, 0.5))
        # except:
        #     auroc =  0.5
        # print(auroc)
        accuracy = accuracy_score(true.to("cpu"), pred.to("cpu"))
        print(accuracy)



        # torch.save(model, f"save/binary_all_features/{Class_to_eval}_{filt}_{NN_size}_{epochs}_{group_holdout}.pt")
